# Data Mining project: Discover and describe areas of interest and events from geo-located data

## 1. Import Dataset and Libraries

In [1]:
# load pandas to deal with the data
import pandas as pd
# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
# load data from table file where entries are separated with a space
data = pd.read_table("flickr_data2.csv", sep=",", low_memory=False)

data.columns = data.columns.str.strip()

print(data.columns)
print(data.info())
print(data.describe())
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'flickr_data2.csv'

## Perform Exploratory Data Analysis
First, we will explore the most common **data quality issues**:
* missing-vals
* duplicates

Second, we will use [**descriptive statistics**](#desc-stats) to have get a statistical summary of the data. 

We will then use [**data visualisaiton**](#data-vis) to get a better understanding of the data.

### Wrong type values

In [3]:
# Affiche la taille initiale
print(f"Initial: {len(data)}")
print(data.iloc[98811])

# Dictionnaire des règles de validation
validation_rules = {
    'lat': lambda x: pd.api.types.is_number(x) and 45.69804 <= x <= 45.79972,
    'long': lambda x: pd.api.types.is_number(x) and 4.76075 <= x <= 4.93901,
    'date_taken_minute': lambda x: pd.api.types.is_number(x) and 0 <= x <= 59,
    'date_taken_hour': lambda x: pd.api.types.is_number(x) and 0 <= x <= 23,
    'date_taken_day': lambda x: pd.api.types.is_number(x) and 1 <= x <= 31,
    'date_taken_month': lambda x: pd.api.types.is_number(x) and 1 <= x <= 12,
    'date_taken_year': lambda x: pd.api.types.is_number(x) and 1900 <= x <= 2100,
    'date_upload_minute': lambda x: pd.api.types.is_number(x) and 0 <= x <= 59,
    'date_upload_hour': lambda x: pd.api.types.is_number(x) and 0 <= x <= 23,
    'date_upload_day': lambda x: pd.api.types.is_number(x) and 1 <= x <= 31,
    'date_upload_month': lambda x: pd.api.types.is_number(x) and 1 <= x <= 12,
    'date_upload_year': lambda x: pd.api.types.is_number(x) and 1900 <= x <= 2100,
}

# Fonction de nettoyage des colonnes
def clean_column(dataframe, column_name, validation_func):
    dataframe[column_name] = dataframe[column_name].apply(
        lambda x: x if validation_func(x) else np.nan
    )

# Appliquer les règles de validation à chaque colonne
for column, rule in validation_rules.items():
    if column in data.columns:
        clean_column(data, column, rule)

print(data.iloc[98811])
data.head()

Initial: 420240
id                                                           7387935070
user                                                       68256211@N06
lat                                                            45.76869
long                                                           4.843872
tags                  portrait,girl,canon,spring,lyon,parade,fille,p...
title                                                    Gay Pride 2012
date_taken_minute                                                  58.0
date_taken_hour                                                      15
date_taken_day                                                       16
date_taken_month                                                      6
date_taken_year                                                    2012
date_upload_minute                                                    2
date_upload_hour                                                     20
date_upload_day                                 

,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15.0,28.0,2.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
1,4394748717,35853470@N00,45.753270,4.862953,NaN,NaN,51.0,17.0,28.0,2.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17.0,28.0,2.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20.0,28.0,1.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20.0,28.0,1.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN


### Missing Values

To check the missing values, several approaches can be used:

1. The `info()` mwthods provides a summary of a dataframe in terms of the types of values, non-null values and memory usage. Thus, by comparing the number of non-null values of each column with the total number of entries, one can have an idea of missing values.
2. Using the [`isna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html) method. By summing the resulting values, we obtain the number of null values for each column.
3. To get the rows with any missing values, you can use `isna()` followed by `any(axis=1)`.

In [4]:
print(f"Initial: {len(data)}")
# remove rows with missing values on the columns id, lat, and long
data_cleaned_missing_values = data.dropna(subset=['id', 'lat', 'long'])
print(f"After removing missing values: {len(data_cleaned_missing_values)}")

Initial: 420240
After removing missing values: 355491


### Removing duplicates

In [6]:
# remove duplicates
print(f"Initial: {len(data_cleaned_missing_values)}")
print(data_cleaned_missing_values.duplicated().sum())
data_cleaned_duplicates = data_cleaned_missing_values.drop_duplicates(subset=['id', 'lat', 'long'],keep='first')
# show the stats
print(f"After removing duplicates: {len(data_cleaned_duplicates)}")

Initial: 355491
213891
After removing duplicates: 141596


### Descriptive Statistics

To obtain the statistical summary of the dataframe, we can use [`describe()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html). For different columns, it displays the count, the average value, the standard deviation, the min and max values, percentiles. 
By default, in mixed data types DataFrames, it displays the values for quantative data only:

In [7]:
data_cleaned_duplicates.describe()

,id,lat,long,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
count,1.415960e+05,141596.000000,141596.000000,141552.000000,141566.000000,141595.000000,141571.000000,141548.000000,0.0,0.0,141594.000000,141569.000000,141549.000000,46.000000,0.0,1.0
mean,2.018108e+10,45.761139,4.836779,29.386063,14.862587,15.091006,7.116387,2014.058065,NaN,NaN,15.347162,6.852425,2014.419501,1928.086957,NaN,2012.0
std,1.373251e+10,0.014849,0.020774,17.450493,4.990502,8.694537,3.406928,3.108261,NaN,NaN,8.448202,3.505216,2.766905,411.524041,NaN,NaN
min,3.066675e+08,45.698056,4.760769,0.000000,0.000000,1.000000,1.000000,1926.000000,NaN,NaN,1.000000,1.000000,2009.000000,12.000000,NaN,2012.0
25%,8.046895e+09,45.757525,4.826516,14.000000,12.000000,8.000000,4.000000,2012.000000,NaN,NaN,8.000000,4.000000,2012.000000,2013.000000,NaN,2012.0
50%,1.582660e+10,45.762366,4.832997,29.000000,15.000000,14.000000,7.000000,2014.000000,NaN,NaN,15.000000,7.000000,2014.000000,2015.000000,NaN,2012.0
75%,3.148142e+10,45.768595,4.844010,44.000000,18.000000,23.000000,10.000000,2017.000000,NaN,NaN,23.000000,10.000000,2017.000000,2016.000000,NaN,2012.0
max,4.914809e+10,45.799710,4.938987,59.000000,23.000000,31.000000,12.000000,2019.000000,NaN,NaN,31.000000,12.000000,2019.000000,2019.000000,NaN,2012.0


## Prepare data for clustering

First, we will droping the columns user, tag and title because they are not necessary for geographic clustering

In [8]:
df_clustering = data_cleaned_duplicates.drop(columns=['user'])
df_clustering = df_clustering.drop(columns=['tags'])
df_clustering = df_clustering.drop(columns=['title'])

df_clustering.head()

,id,lat,long,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,date_taken_year,date_upload_minute,date_upload_hour,date_upload_day,date_upload_month,date_upload_year,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,4395181099,45.754858,4.821710,11.0,15.0,28.0,2.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
1,4394748717,45.753270,4.862953,51.0,17.0,28.0,2.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
2,4394694699,45.760655,4.846564,29.0,17.0,28.0,2.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
3,4394803790,45.784000,4.874072,15.0,20.0,28.0,1.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN
4,4394803554,45.784000,4.874072,10.0,20.0,28.0,1.0,2010.0,NaN,NaN,28.0,2.0,2010.0,NaN,NaN,NaN


Let's apply a [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Recall, that for a given value `x`, a standard score is given by $z = \frac{x - mean(\mathbf{x})}{std(\mathbf{x})}$ 

In [10]:
# scaler
from sklearn.preprocessing import StandardScaler

In [10]:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_clustering)
# show
print(scaled_data)
# create a DataFrame
scaled_data_df = pd.DataFrame(data=scaled_data, columns=df_clustering.columns)
scaled_data_df.head()

print(len(scaled_data_df))

[[-1.14953145 -0.42300291 -0.72538134 ...         nan         nan
          nan]
 [-1.14956294 -0.52994641  1.25994429 ...         nan         nan
          nan]
 [-1.14956687 -0.0326053   0.47102249 ...         nan         nan
          nan]
 ...
 [ 2.09820947  1.39564253  1.85987915 ...         nan         nan
          nan]
 [ 1.79453936  0.50918336  0.17921433 ...         nan         nan
          nan]
 [ 1.80695534  0.72145406 -0.17637572 ...         nan         nan
          nan]]
141596


C:\Users\Malte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\Malte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
C:\Users\Malte\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


## Hierarchical Clustering

In [ ]:
# scaler
from sklearn.preprocessing import StandardScaler

In [8]:
# load data from table file where entries are separated with a ','
dataHierarch = pd.read_table("datasetCleaned.csv", sep=",", low_memory=False)

# remove leading and trailing whitespaces from column names
dataHierarch.columns = dataHierarch.columns.str.strip()
#remove entries with same coordinates
print(len(dataHierarch))
dataHierarch=dataHierarch.drop_duplicates(subset=['lat', 'long'])
print(len(dataHierarch))


168097
66819


In [23]:
# Limiter aux coordonnées de Lyon et diviser dans deux zones geographiques
lyon_bounds = {
    'lat_min': 45.709, #original: 45.709
    'lat_max': 45.762, #original: 45.80
    'lon_min': 4.79, #original:  4.79
    'lon_max': 4.90 #original: 4.90
}

mask = (
    (dataHierarch['lat'] >= lyon_bounds['lat_min']) &
    (dataHierarch['lat'] <= lyon_bounds['lat_max']) &
    (dataHierarch['long'] >= lyon_bounds['lon_min']) &
    (dataHierarch['long'] <= lyon_bounds['lon_max'])
)
dataHierarch1 = dataHierarch[mask]
# Indizes von dataHierarch1 erhalten
indices_to_exclude = dataHierarch1.index

# Zeilen auswählen, die nicht in den Indizes von dataHierarch1 enthalten sind
dataHierarch2 = dataHierarch.loc[~dataHierarch.index.isin(indices_to_exclude)]
print(len(dataHierarch))
print(len(dataHierarch1))
print(len(dataHierarch2))

# Préparer les données pour le clustering
X1 = dataHierarch1[['lat', 'long']].values
X2 = dataHierarch2[['lat', 'long']].values

# Normaliser les données
scaler = StandardScaler()
X_scaled1 = scaler.fit_transform(X1)
X_scaled2 = scaler.fit_transform(X2)

62397
30424
31973


In [21]:
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

In [28]:
pip install folium

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/110.5 kB ? eta -:--:--
   ------- ------------------------------- 20.5/110.5 kB 640.0 kB/s eta 0:00:01
   ---------------------------------------- 110.5/110.5 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/88.4 kB ? eta -:--:--
   ---------------------------------------- 88.4/88.4 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: C:\Users\Malte\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [29]:
import folium

In [26]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians
import colorsys
from collections import Counter
from scipy.spatial import ConvexHull

In [32]:
import webbrowser
import os

In [45]:
def get_cluster_tags(df, cluster_id, mots_exclus):
    """Trouve le tag le plus représentatif pour un cluster"""
    cluster_data = df[df['cluster'] == cluster_id]
    all_tags = []
    
    for tags_str in cluster_data['tags'].fillna(''):
        tags_list = tags_str.lower().split(',')
        for tag in tags_list:
            tag = tag.strip()
            if tag not in mots_exclus and not any(c.isdigit() for c in tag):
                subtags = tag.replace('_', ' ').replace('-', ' ').split()
                all_tags.extend(subtags)
    
    tag_counts = Counter(all_tags)
    most_common_tags = [(tag, count) for tag, count in tag_counts.most_common(10)
                       if len(tag) > 2 and ' ' not in tag]
    
    return most_common_tags[0][0] if most_common_tags else f"Cluster{cluster_id}"


def cluster_and_visualize(data1, data2, lyon_bounds, mots_exclus):
    # Daten vorbereiten
    data1 = data1[['lat', 'long', 'tags']].copy()
    data2 = data2[['lat', 'long', 'tags']].copy()

    # Clustering für beide Datensets durchführen
    scaler = StandardScaler()
    clusters_per_set = 25

    # Erstes Datenset clustern
    X1 = scaler.fit_transform(data1[['lat', 'long']])
    cluster_model1 = AgglomerativeClustering(n_clusters=clusters_per_set, linkage='ward')
    data1['cluster'] = cluster_model1.fit_predict(X1)

    # Zweites Datenset clustern
    X2 = scaler.fit_transform(data2[['lat', 'long']])
    cluster_model2 = AgglomerativeClustering(n_clusters=clusters_per_set, linkage='ward')
    data2['cluster'] = cluster_model2.fit_predict(X2)

    # Cluster-IDs des zweiten Datensets anpassen, um IDs 0-49 zu haben
    data2['cluster'] += clusters_per_set

    # Beide Datensets zusammenfügen
    data_combined = pd.concat([data1, data2], ignore_index=True)

    # Cluster-Tags berechnen
    cluster_tags = {}
    for cluster_id in range(50):
        cluster_tags[cluster_id] = get_cluster_tags(data_combined, cluster_id, mots_exclus)

    representative_points = (
    data_combined.groupby('cluster')[['lat', 'long']].mean().reset_index()
    )


    # Karte erstellen
    print("Erstellen der Karte...")
    m = folium.Map(location=[data_combined['lat'].mean(), data_combined['long'].mean()], zoom_start=13)

    # Rechteck der Lyon-Grenzen hinzufügen
    bounds = [[lyon_bounds['lat_min'], lyon_bounds['lon_min']],
              [lyon_bounds['lat_max'], lyon_bounds['lon_max']]]
    folium.Rectangle(bounds=bounds, color='red', weight=2, fill=False, popup='Zone d\'étude').add_to(m)

    # Farben für Cluster generieren
    colors = []
    for i in range(50):
        hue = i / 50
        rgb = colorsys.hsv_to_rgb(hue, 0.8, 0.8)
        color = '#{:02x}{:02x}{:02x}'.format(
            int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255))
        colors.append(color)

       # Repräsentative Punkte auf die Karte setzen
    for _, row in representative_points.iterrows():
        cluster_id = int(row['cluster'])
        popup_content = f"""
        <div style="min-width: 200px;">
        Cluster: {cluster_tags[cluster_id]}
        </div>
        """
        
        folium.CircleMarker(
            location=[row['lat'], row['long']],
            radius=8,
            color=colors[cluster_id],
            fill=True,
            popup=popup_content,
            fill_opacity=0.7
        ).add_to(m)

    # Karte speichern und öffnen
    print("Karte speichern...")
    m.save('carte_clusters.html')
    webbrowser.open(f'file:///{os.path.abspath("carte_clusters.html")}')

    return data1, data2


# Beispiel-Daten und Parameter
lyon_bounds = {
    'lat_min': 45.709,
    'lat_max': 45.80,
    'lon_min': 4.79,
    'lon_max': 4.90
}

mots_exclus = ['unknown', 'lyon', '', 'france', 'europe', 'nuit', 'streetphotography', 'french', 'rhônealpes']

# Hauptfunktion ausführen
data_hierarchical1, data_hierarchical2 = cluster_and_visualize(dataHierarch1, dataHierarch2, lyon_bounds, mots_exclus)

Erstellen der Karte...
Karte speichern...


In [48]:
# Zugriff auf Clusterinformationen von data_hierarchical1
print(data_hierarchical2.head)


<bound method NDFrame.head of               lat      long  \
3       45.784000  4.874072   
7       45.774662  4.834005   
8       45.762328  4.827547   
9       45.762059  4.822654   
10      45.767328  4.833362   
...           ...       ...   
168080  45.785599  4.813400   
168081  45.784841  4.812855   
168082  45.781863  4.875416   
168089  45.768700  4.840502   
168096  45.771852  4.833115   

                                                     tags  cluster  
3            nin,nineinchnails,gift,screening,toiou,avott       44  
7       lyon,streetphotography,rue,montblanc,gens,mont...       25  
8                           france,lyon,lesphotosdevoyage       27  
9                 france,lyon,fourvière,lesphotosdevoyage       42  
10                                      lesphotosdevoyage       32  
...                                                   ...      ...  
168080                                                NaN       38  
168081                                       